In [47]:
pip install pandas selenium webdriver_manager requests bs4 openai yt_dlp openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
from bs4 import BeautifulSoup
import requests

# URL you want to scrape
url = 'https://subslikescript.com/series/Shark_Tank-1442550'

# Fetch the content from the URL
response = requests.get(url)
html_content = response.text

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find all anchor tags and extract the 'href' attribute
urls = [a.get('href') for a in soup.find_all('a', href=True)]

# Filter the URLs to only include the ones that start with '/series/Shark_Tank'
urls = [f'https://subslikescript.com{link}' for link in urls if link.startswith('/series/Shark_Tank')]
# Print the list of URLs
for link in urls:
    print(link)

https://subslikescript.com/series/Shark_Tank-1442550/season-12/episode-1-Episode_121
https://subslikescript.com/series/Shark_Tank-1442550/season-12/episode-2-Episode_122
https://subslikescript.com/series/Shark_Tank-1442550/season-12/episode-6-Episode_126
https://subslikescript.com/series/Shark_Tank-1442550/season-12/episode-9-Episode_129
https://subslikescript.com/series/Shark_Tank-1442550/season-12/episode-11-Episode_1211
https://subslikescript.com/series/Shark_Tank-1442550/season-12/episode-23-Episode_1223
https://subslikescript.com/series/Shark_Tank-1442550/season-12/episode-24-Episode_1224
https://subslikescript.com/series/Shark_Tank-1442550/season-12/episode-25-Episode_1225
https://subslikescript.com/series/Shark_Tank-1442550/season-13/episode-4-Episode_134
https://subslikescript.com/series/Shark_Tank-1442550/season-13/episode-5-Episode_135
https://subslikescript.com/series/Shark_Tank-1442550/season-13/episode-6-Episode_136
https://subslikescript.com/series/Shark_Tank-1442550/seas

In [49]:
extract_list = []

for link in urls:
    response = requests.get(link)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')
    script = soup.find('div', class_='full-script')

    if script:
        for br in script.find_all("br"):
            br.replace_with("\n")
    
        script_text = script.get_text() if script else 'No script found'
        extract_list.append({'url': link, 'script': script_text})

In [50]:
import pandas as pd

df = pd.DataFrame(extract_list)
df.to_excel('shark_tank_scripts.xlsx', index=False)

In [51]:
for i in range(0, len(df)):
    filename = df['url'][i].split('/')[-1]
    with open(f'raw_web_scrape/sn_{i}_shark_tank_script_{filename}.txt', 'w',encoding='utf-8') as f:
        f.write(df['script'][i])

In [52]:
sys_context="""
You are a helpful transcript bot.

Do Not Summarize.

leave the text as is.
"""

In [89]:
instructions = """
Please read this transcript and return the product names being pitched in the episode. There are multiple products being pitched in the episode. Please return the product names as a list.

I need you to identify the first line in the transcript that starts each product segment, for example "First into the Tank" or "Next into the Tank". The line itself please as it needs to be used to identify the product.

Return as follows in a readable json format: 

[{"Product": <Product Name>,"First_Line": <Transcript Line>}]

Return only the list
"""

In [86]:
from openai import OpenAI

verbose = False

def call_api(sys_context, user_input,tempreature=0.2,max_completion_tokens=4096):
    client = OpenAI()
    if verbose:
        print("Calling OpenAI API")

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature = tempreature,
        max_completion_tokens=max_completion_tokens,
        # top_p = 0.1,
        messages=[
            {"role": "system", "content": sys_context},
            {
                "role": "user","content": user_input
                # "content": instructions + extract_list[0]['script']
            }
        ]
    )
    if verbose:
        print(completion.usage)

    return completion.choices[0].message.content

In [80]:
def line_breaker(data, lines): 
    line = [l for l in lines if l and l.strip() !='Narrator:']
    line_break = []
    for d in data:
        for i, l in enumerate(line):
            if l in (d['First_Line']):
                search_line = []
                for k in range(2):
                    search_line.append(line[i+k])
                # print(search_line)
                # print(d['First_Line'])
                if " ".join(search_line) in d['First_Line']:
                    line_break.append(i)
                    break

    line_break.append(len(line))
    # print(line_break)

    transcripts = []

    for i in range(len(line_break)-1):
        if verbose:
            print(line_break[i], line_break[i+1])
            print(line[line_break[i]:line_break[i+1]])
        
        part = line[line_break[i]:line_break[i+1]]

        #logging
        with open('log.txt', 'a',encoding='utf-8') as f:
            f.write(f"{line_break[i]} {line_break[i+1]}\n")
            f.write(f"{part[0]} | {part[-1]}\n\n")

        transcripts.append((line[line_break[i]:line_break[i+1]]))

    return transcripts

In [90]:
import json

def json_converter(input_text):

    # Remove the markdown formatting:
    # Split the text into lines and remove the first and last lines if they are markdown fences.
    lines = input_text.strip().splitlines()
    if lines[0].startswith("```"):
        lines = lines[1:]
    if lines[-1].startswith("```"):
        lines = lines[:-1]

    # Join the remaining lines to reconstruct the JSON string.
    json_str = "\n".join(lines)

    # Convert the JSON string into a Python object (list of dictionaries)
    data = json.loads(json_str)

    return data

# Print the resulting Python object
# data = json_converter(splitter_reply)

[{'Product': 'Pluto Pillow', 'First_Line': 'First into the Tank'},
 {'Product': 'Frobert',
  'First_Line': 'Next up is a new iteration of a favorite sport.'},
 {'Product': 'Fling Golf',
  'First_Line': 'Next up is a new iteration of a favorite sport.'},
 {'Product': "Float'N'Grill",
  'First_Line': 'Next into the Tank is a better way to grill in an unlikely place.'},
 {'Product': 'The Original Stretchlace',
  'First_Line': 'Next up is an accessory that makes shoes more functional.'}]

In [ ]:
# transcripts = line_breaker(data, script.split('\n'))

30 713
['Narrator: First into the Tank', 'is what the entrepreneur hopes', 'is a better and more', 'affordable experience', 'when shopping for intimates.', '♪♪', 'Hi, Sharks. My name is Elly.', "I'm from San Francisco,", 'California,', 'and I am seeking $500,000', 'in exchange for 5%', 'of my company...', '‐Hello. ‐Wow. Wow.', '‐...BootayBag. ‐Ha.', 'BootayBag is an affordable,', 'comfortable, and convenient way', 'for women to shop for underwear.', 'I started BootayBag because', 'I was sick of going to the mall,', 'the semi‐annual sales,', 'the clearance bins,', 'the sales clerks.', 'I wanted to give women', 'that sexy feeling', 'of sliding on', 'a new pair of underwear', 'without all the hassle', 'or expensive price tag.', "So, here's how it works.", 'You select how many pairs', 'you want to receive in a month,', 'your size, your style option,', "and for just $10, we'll deliver", 'your new favorite undies', 'all without leaving', 'the comfort of your couch.', 'Our mission is to not o

In [ ]:
# import os

# verbose = True
# for j, transcript in enumerate(transcripts):
#     # if file exists, skip
#     if not os.path.exists(f'raw_data/shark_tank_transcript_raw_{i}_{data[j]['Product']}.txt'):
#         with open(f'raw_data/shark_tank_transcript_raw_{i}_{data[j]['Product']}.txt', 'w',encoding='utf-8') as f:
#             f.write(str(transcript))

In [ ]:
transcript_instructions = """
Below is the extracted subtitles of a Shark Tank episode. 

Note that the transcript is split between the Shark and the Entrepreneur. The Entrepreneur is the person who is pitching the product and the Shark is an angel investor who is looking to invest in the product.

Please analyse each line and determine if the speaker is a Entrepreneur or Shark.

Please return this in an json readable format as follows:

REMEMBER TO READ ALL THE LINES

Only provide information that is explicitly mentioned in the text. Leave the text as is.


[{"Speaker":<Entrepreneur/Shark>,
"Text": <Utterance>},
{"Speaker":<Entrepreneur/Shark>,
"Text": <Utterance>}]

"""

In [ ]:
import os

failed = []
for i, raw in enumerate(extract_list):
    try:
        verbose = False
        print(raw['url'])
        data = json_converter(call_api(sys_context, instructions + raw['script']))
        transcripts = line_breaker(data, raw['script'].split('\n'))
        for j, transcript in enumerate(transcripts):
            # if file exists, skip
            if not os.path.exists(f'raw_data/shark_tank_transcript_raw_{i}_{data[j]['Product']}.txt'):
                with open(f'raw_data/shark_tank_transcript_raw_{i}_{data[j]['Product']}.txt', 'w',encoding='utf-8') as f:
                    f.write(str(transcript))
            # if file exists, skip
            if os.path.exists(f'Prelim_Data_Split/shark_tank_transcript_{i}_{data[j]['Product']}.txt'):
                continue
            print(data[j]['Product'])
            user_input = transcript_instructions + "\n".join(transcript)
            output =  call_api(sys_context, user_input, tempreature=0.5,max_completion_tokens=8192)
            output = json_converter(output)
            with open(f'Prelim_Data_Split/shark_tank_transcript_{i}_{data[j]['Product']}.txt', 'w',encoding='utf-8') as f:
                f.write(str(output))
    except Exception as e:
        print((i, j, e))
        failed.append((i, j, e))
        continue

with open(f"failed.txt", 'w') as f:
    for item in failed:
        f.write(f"{item}\n")

https://subslikescript.com/series/Shark_Tank-1442550/season-12/episode-1-Episode_121
https://subslikescript.com/series/Shark_Tank-1442550/season-12/episode-2-Episode_122
Pooch Paper
P-nuff Crunch
https://subslikescript.com/series/Shark_Tank-1442550/season-12/episode-6-Episode_126
https://subslikescript.com/series/Shark_Tank-1442550/season-12/episode-9-Episode_129
Basepaws
https://subslikescript.com/series/Shark_Tank-1442550/season-12/episode-11-Episode_1211
https://subslikescript.com/series/Shark_Tank-1442550/season-12/episode-23-Episode_1223
Boost Oxygen
(5, 1, JSONDecodeError('Expecting property name enclosed in double quotes: line 341 column 25 (char 29519)'))
https://subslikescript.com/series/Shark_Tank-1442550/season-12/episode-24-Episode_1224
(6, 1, IndexError('list index out of range'))
https://subslikescript.com/series/Shark_Tank-1442550/season-12/episode-25-Episode_1225
https://subslikescript.com/series/Shark_Tank-1442550/season-13/episode-4-Episode_134
https://subslikescript.